# ELO Rating by Clazy Chen

## ITTF formats

### event.json
Each `event.json` is formatted as `[[dicts]]`, where in each dict:
- vw_tournaments___id_raw -> uid
- vw_tournaments___tournament_id_raw -> id
- vw_tournaments___yr_raw -> year
- vw_tournaments___tournament_raw -> name
- vw_tournaments___type -> type
- vw_tournaments___kind -> kind
- vw_tournaments___organizer -> organizer
- vw_tournaments___tour_start_raw -> start (YYYY-MM-DD)
- vw_tournaments___tour_end_raw -> end (YYYY-MM-DD)

### match.json
Each `match.json` is formatted as `[[dicts]]`, where in each dict:
- vw_matches___id_raw -> id
- vw_matches___tournament_id_raw -> event_id
- vw_matches___tournament_id -> event_name (ITTF's terrible naming)
- vw_matches___player_a_id_raw -> player_a_id
- vw_matches___name_a_raw -> player_a_name (can also be looked up with id)
- vm_matches___assoc_a -> player_a_assoc (when this event took place)
- vm_matches___player_b -> for MD, WD, XD, the partner of player_a
- vm_matches___player_x -> the opponent of player_a
- vm_matches___player_y -> for MD, WD, XD, the partner of player_x
- vm_matches___stage_raw -> Qualification, R32, R16, QF, SF, F
- vm_matches___res_raw -> the result of the match, formatted "A - X"
- vm_matches___games_raw -> the results of each games, formatted "A-X A-X A-X" (may have 0:0 or space suffix)

### player.json
Each `player.json` is formatted as `[[dicts]]`, where in each dict:
- vw_profiles___player_id_raw -> id
- vw_profiles___player_id -> "name (assoc)"
- vw_profiles___name_raw -> "name (#id)"
- vm_profiles___gender_raw -> M / W
- vm_profiles___assoc_raw -> association (at present)
- vm_profiles___profile_raw -> we will analyze this field to get "YoB" and "Style", e.g., `"<img src='https://results.ittf.link/images/stories/flags/USA.png' width='43' height='29'><br/>MA: USA<br/>Gender: Male<br/>YoB: 1991<br/>Age: 33<br/>Style: Left-Hand Attack (ShakeHand)<br/>Rank: 203 | Week: 15/2025 (Apr 8th, 2025)<br/>Career Best**: 135\n| Week: 20/2024 (Apr 8th, 2025)<br/><br/>Best result***: <br/>2025 - ITTF Pan American Cup, San Francisco (USA) | R16"`

## Download ITTF files

It takes some time to download ITTF files (larger than 1GB) if you want to build a different rankings, or you can just use the files with a converted format (easy to use) in my repo. Here is some instructions to download ITTF files.

- First, you should log in the ITTF website, F12 in your explorer to get your cookies
- Use your cookies to visit the query API provided by ITTF. Use `format=json` if you do not want to analyze the raw HTML data.
- You can first download all events, then all matches (using the event id), and finally players (using the player id in all matches).
- You can run this jupyter notebook to convert the ITTF format to my format, or you may prefer your own format.

## My format
I remove all redundant fields in the ITTF json. The data in my format is stored in two files: matches and players. For matches, I group them in end-dates, and rename the files with YYYYMMDD.json. In each json there is a list of dicts, and each dict represents a event which ended at the YYYYMMDD date. It is convenient for me to use these data to compute the ratings and rankings at a certain date then.

Only single matches (MS or WS) is recorded in my format, as my algorithm is not suitable for double matches. Table tennis is significantly different from badminton that two strong players can always form a strong pair even if they have not play together in an ITTF/WTT double's match before. For example, SUN Yingsha and KUAI Man - I believe they can beat many players in double's matches - but this pair has no elo rating at all.

On the other hand, all players are stored in a one json. The player.json is also a list of dicts, and each dict represents a player.

### format of events
- id
- name
- weight (from the category)
- time (the end-date in YYYYMMDD, should be same as the filename)
- match (list)
  - player_a_id
  - player_x_id
  - weight (computed by result)
  - result (in "A:X")
  - games (in "a:x a:x a:x")

My algorithm do not take the results of each game in consideration, because table tennis is an 11-point game, when it comes to 6:0 / 7:0, many players may choose to "abandon" this game to conserve their stamina, or adopt a more aggressive style to observe their opponents' response. A 11:1 does not mean that there is a huge gap in strength between the two players.

### format of players
- id
- name
- sex (M/W)
- history (list) (for players who have changed their association, like ZHU Yuling)
  - until
  - assoc
- yob (year of born)
- hand (left or right)
- style (attacker or defense)
- grip (shakehand or penhold)

## Directory structure

In our project, the files in ITTF formats are in (not uploaded to GitHub):
- events are stored in `events/events_xxx.json`, where `xxx = 0, 1, 2, ...` (same hereafter)
- matches are stored in `matches/matches_xxx.json`.
- players are stored in `players/[playerid].json`.

While the files in my formats are in:
- events (with matches) are stored in `data/events/YYYYMMDD.json`.
- players are stored in `data/players.json`.

# Workflow

## Overview

The workflow has four steps.
1. Load local files
2. Update data (download from the ITTF website)
3. Update & Store local files
4. Run the algorithm to generate rankings

If you do not want to update data (e.g. use your own data instead of ITTF data), please omit Step 2 and 3.

## Step 1 - Read the files in my formats

First, we should read the files in my formats, which represents the data which have been processed in the last update. This time we only need to download the data for new events (and corresponding matches or players). 

### 1.1 Define structures

In [1]:
using JSON
using Dates

struct Match
    player_a_id::Int
    player_x_id::Int
    player_a_assoc::String
    player_x_assoc::String
    weight::Float32
    result::String
    games::String
end

struct Event
    id::Int
    name::String
    weight::Float32
    time::Date
    match::Vector{Match}
end

struct Player
    id::Int
    name::String
    sex::String
    history::Dict{String, Date}
    yob::Int
    hand::String
    style::String
    grip::String
end

### 1.2 Read events and matches

In [2]:
function read_events_from_files()
    # Create data/events directory if it doesn't exist
    if !isdir("data/events")
        mkdir("data/events")
        println("Created data/events directory")
    end
    
    # Store events in a Dict with Date keys and Vector{Event} values
    events_by_date = Dict{Date, Vector{Event}}()
    
    # Get all JSON files from data/events directory
    event_files = filter(f -> endswith(f, ".json"), readdir("data/events", join=true))
    
    for file_path in event_files
        open(file_path, "r") do file
            data = JSON.parse(read(file, String))
            
            # Process each event in the file
            for event_data in data
                if haskey(event_data, "id") && haskey(event_data, "name") && 
                   haskey(event_data, "weight") && haskey(event_data, "time") && 
                   haskey(event_data, "match")
                    
                    # Parse match data
                    matches = Match[]
                    for match_data in event_data["match"]
                        if haskey(match_data, "player_a_id") && haskey(match_data, "player_x_id") &&
                           haskey(match_data, "player_a_assoc") && haskey(match_data, "player_x_assoc") &&
                           haskey(match_data, "weight") && haskey(match_data, "result") && 
                           haskey(match_data, "games")
                            
                            push!(matches, Match(
                                match_data["player_a_id"],
                                match_data["player_x_id"],
                                match_data["player_a_assoc"],
                                match_data["player_x_assoc"],
                                match_data["weight"],
                                match_data["result"],
                                match_data["games"]
                            ))
                        end
                    end
                    
                    # Create event object
                    event_date = Date(event_data["time"], "yyyymmdd")
                    event = Event(
                        event_data["id"],
                        event_data["name"],
                        event_data["weight"],
                        event_date,
                        matches
                    )
                    
                    # Add to date-indexed dictionary
                    if !haskey(events_by_date, event_date)
                        events_by_date[event_date] = Event[]
                    end
                    push!(events_by_date[event_date], event)
                end
            end
        end
    end
    
    return events_by_date
end

# Read all events
events = read_events_from_files()
println("Read events for $(length(events)) dates from data/events directory")

Read events for 1290 dates from data/events directory


### 1.3 Read player file

In [3]:
# Read Player information
function read_players_from_file(file_path::String = "data/players.json")
    players_dict = Dict{Int, Player}()
    
    if isfile(file_path)
        open(file_path, "r") do file
            player_data = JSON.parse(file)
            
            for player in player_data
                # Process association information
                associations = Dict{String, Date}()
                if haskey(player, "history")
                    for (assoc, date_str) in player["history"]
                        associations[assoc] = Date(date_str)
                    end
                end
                
                # Create Player object and add to dictionary with player_id as key
                player_id = player["id"]
                players_dict[player_id] = Player(
                    player_id,
                    player["name"],
                    get(player, "sex", "U"),
                    associations,
                    get(player, "yob", 0),
                    get(player, "hand", "Unknown Handness"),
                    get(player, "style", "Unknown Style"),
                    get(player, "grip", "Unknown Grip")
                )
            end
        end
    else
        println("Warning: Player data file $file_path not found")
    end
    
    return players_dict
end

# Read all players
players = read_players_from_file()
println("Read $(length(players)) players from data/players.json file")

Read 29079 players from data/players.json file


## Step 2 - Update data

### 2.1 Use the COOKIES
**Warning: You should use your own cookies (instead of mine).** 

An outdated token may not work.

In [3]:
using HTTP

const COOKIES = Dict(
    "joomla_user_state" => "logged_in",
    "jbcookies" => "yes",
    "7f39b4f271ec48e98fc313e8ebd29f8f" => "75dba7e749de55256e57abb54d55c00b",
    "_ga" => "GA1.1.364169816.1702376315"
)

println("Cookies are set.")

Cookies are set.


### 2.2 Update events

Convert the ITTF format data into the Event structure. 

#### 2.2.1 Compute the weight

The **weight** of an event is computed by its type. Different types of events have different weights, because major events are more intense and receive greater attention from players. Many players may use regular WTT events as practice while giving their all in major events. On the other hand, ITTF/WTT points are also important. I assign weights to each event based on the above two aspects.

Due to my limited understanding of historical events, these weight designs may not be sufficiently reasonable and may be further optimized in the future. Below is the currently adopted weight table (main types only).

| Event Type                          | Weight |
| ----------------------------------- | ------ |
| Olympic Games                       |   3.0  |
| World Championships                 |   2.5  |
| World Cup                           |   2.0  |
| Asian/European Games                |   1.6  |
| Asian/European Championships        |   1.5  |
| Asian/European(Top-16) Cup          |   1.5  |
| ITTF/WTT Finals                     |   1.5  |
| WTT Grand Smash                     |   1.4  |
| WTT Champions & ITTF Platium Open   |   1.3  |
| WTT Star Contender & ITTF Open      |   1.2  |
| WTT Contender & ITTF Challenge Plus |   1.1  |
| WTT Feeder & ITTF Challenge         |   1.0  |

In [14]:
# Check if an event is a youth event
function is_youth(name::String)
    youth_patterns = [
        "Youth", "youth",
        "Cadet", "cadet",
        "Junior", "junior",
        "U21", "U-21", "YOG",
        "U15", "U-15",
        "U18", "U-18"
    ]
    return any(pattern -> occursin(pattern, name), youth_patterns)
end

# Helper function to check if event is Asian/European
function is_major_continental(name::String)
    return any(region -> occursin(region, name), ["Asian", "European", "Asia", "Europe"])
end

# Pattern matching for different event types
function match_weight(::Val{:Olympic_Games}, name::String)
    if occursin("Qualification", name) || occursin("Road", name)
        return 0.2
    elseif is_youth(name)
        return 1.0
    else
        return 3.0
    end
end

function match_weight(::Val{:Youth_Olympic_Games}, name::String)
    return 1.0
end

function match_weight(::Val{:Youth_Olympic_Games_Qualification}, name::String)
    return 0.2
end

function match_weight(::Val{:WTTC}, name::String)
    return 2.5
end

function match_weight(::Val{:World_Cup}, name::String)
    return 2.0
end

function match_weight(::Val{:WTT_Finals}, name::String)
    return 1.5
end

function match_weight(::Val{:World_Tour__Pro_Tour}, name::String)
    if occursin("Finals", name)
        return 1.5
    elseif occursin("Platinum", name)
        return 1.3
    else
        return 1.2
    end
end

function match_weight(::Val{:WTT_Champions}, name::String)
    return 1.3
end

function match_weight(::Val{:WTT_Grand_Smash}, name::String)
    return 1.4
end

function match_weight(::Val{:WTT_Youth_Grand_Smash}, name::String)
    return 0.5
end

function match_weight(::Val{:Continental_Games}, name::String)
    if is_youth(name)
        return 0.25
    else
        return is_major_continental(name) ? 1.6 : 0.8
    end
end

function match_weight(::Val{:Continental}, name::String)
    if is_youth(name)
        return is_major_continental(name) ? 0.5 : 0.25
    else
        return is_major_continental(name) ? 1.5 : 0.75
    end
end

function match_weight(::Val{:WTT_Contender_Series}, name::String)
    return occursin("Star", name) ? 1.2 : 1.1
end

function match_weight(::Val{:T2_Diamond}, name::String)
    return 1.0
end

function match_weight(::Val{:WJTTC}, name::String)
    return 0.8
end

function match_weight(::Val{:Challenge}, name::String)
    return occursin("Plus", name) ? 1.1 : 1.0
end

function match_weight(::Val{:Olympic_Qualification}, name::String)
    return 0.6
end

function match_weight(::Val{:World_Youth_Championships}, name::String)
    return 0.8
end

function match_weight(::Val{:World_Cadet_Challenge}, name::String)
    return 0.65
end

function match_weight(::Val{:World_Junior_Circuit}, name::String)
    if occursin("Finals", name)
        return 0.5
    elseif occursin("Platinum", name) || occursin("Golden", name)
        return 0.4
    else
        return 0.3
    end
end

function match_weight(::Val{:WTT_Feeder_Series}, name::String)
    return 1.0
end

function match_weight(::Val{:Multi_sport_events}, name::String)
    # Asian Games special cases
    if occursin("Asian", name) && (occursin("Guangzhou", name) || occursin("Incheon", name))
        return 1.6
    elseif occursin("Pan American", name) && occursin("Guadalajara", name)
        return 0.8
    elseif is_youth(name)
        return 0.2
    else
        return 0.6
    end
end

function match_weight(::Val{:Other_events}, name::String)
    if occursin("Open,", name)
        return is_youth(name) ? 0.3 : 1.0
    elseif occursin("WTTC", name)  # WTTC qualification
        return 0.5
    elseif occursin("Top 10", name)  # European Youth Top 10
        return 0.5
    else
        return is_youth(name) ? 0.2 : 0.6
    end
end

function match_weight(::Val{:WTT_Youth_Contender_Series}, name::String)
    return occursin("Star", name) ? 0.4 : 0.3
end

# Fallback for unknown event types
function match_weight(::Val, name::String)
    return 0.1
end

# Main weight calculation function
function weight(type_::String, name::String)
    # Special cases first
    if occursin("China vs World Team", name) || occursin("China vs. World Team", name)
        return 1.2
    elseif occursin("Tournament of Champions", name)
        return 1.5
    end

    # Convert type string to symbol, replacing spaces and special characters
    type_symbol = Symbol(replace(type_, " " => "_", "/" => ""))
    
    # Main weight calculation using pattern matching
    return match_weight(Val(type_symbol), name)
end

weight (generic function with 1 method)

#### 2.2.2 Parse the data in ITTF format

In [15]:
function convert_ittf_event(raw_event)
    # Extract only the necessary fields from ITTF format
    id = raw_event["vw_tournaments___tournament_id_raw"]
    name = raw_event["vw_tournaments___tournament_raw"]
    time = Date(raw_event["vw_tournaments___tour_end_raw"])
    type = raw_event["vw_tournaments___type"]
    weight_ = weight(type, name)
    
    return Event(id, name, weight_, time, Match[])
end

function process_events(existing_events::Dict{Date, Vector{Event}}, new_events_json::String)
    events = Event[]
    
    data = JSON.parse(new_events_json)
    new_data = [[]]

    # ITTF format has array of arrays
    for event_list in data
        for raw_event in event_list
            # Convert and store each event immediately
            event = convert_ittf_event(raw_event)
            if !haskey(existing_events, event.time) || 
                !any(e -> e.id == event.id, existing_events[event.time])
                push!(new_data[1], raw_event)
                push!(events, event)
            end
        end
    end
    
    return events, new_data
end

process_events (generic function with 1 method)

#### 2.2.3 Download the event data from the ITTF website

In [ ]:
function download_events_from_ittf(events::Dict{Date, Vector{Event}})   
    # Create events directory if it doesn't exist
    events_dir = "events"
    if !isdir(events_dir)
        mkpath(events_dir)
    end

    # Find the next available file number
    next_file_num = 0
    while isfile(joinpath(events_dir, "events_$(next_file_num).json"))
        next_file_num += 1
    end
    
    # Download events from ITTF API
    new_events = Event[]
    offset = 0
    all_new = true
    
    while all_new
        url = "https://www.results.ittf.link/index.php?option=com_fabrik&view=list&listid=27&Itemid=268&format=json&limit27=100&limitstart27=$(offset)"

        success = false
        
        while !success
            try
                response = HTTP.get(url, cookies=COOKIES)
                
                if response.status != 200
                    println("Error: API request failed with status code: $(response.status)")
                    sleep(10)
                    continue
                end
                
                # Process the downloaded JSON file
                downloaded_events, download_json = process_events(events, String(response.body))
                
                # Check if we have any new events
                if isempty(downloaded_events)
                    all_new = false
                    println("No more event data available")
                    break
                else
                    all_new = length(downloaded_events) == 100
                end

                # Add to our collection
                append!(new_events, downloaded_events)

                # Save to file
                target_file = joinpath(events_dir, "events_$(next_file_num).json")
                open(target_file, "w") do io
                    JSON.print(io, download_json)
                end
                println("Save events to $(target_file)")
                next_file_num += 1
                
                success = true
            catch e
                println("Request failed: $(e)")
                sleep(10)
            end
        end
        
        # Move to next batch
        offset += 100
        
        # Rate limiting
        sleep(0.5)
    end

    return new_events
end

new_events = download_events_from_ittf(events)
println("Downloaded $(length(new_events)) new events")

### 2.3 Update matches

Convert the ITTF format data into the Match structure, and push them into the new events. 

#### 2.3.1 Compute the weight

The **weight** of a match is computed by the number of games won by each player. It is calculated by a normal distribution. Here are some approximate values:

| W / L |   3  |   2  |   1  |   0  |
| ----- | ---- | ---- | ---- | ---- |
|   4   | 0.38 | 0.68 | 0.87 | 0.95 |
|   3   |      | 0.33 | 0.61 | 0.81 |
|   2   |      |      | 0.28 | 0.52 |
|   1   |      |      |      | 0.20 | 

In [17]:
using Distributions

function calculate_weight(w::Int, l::Int)
    # Use normal distribution with scale parameter adjusted by winner's games
    norm_dist = Normal(0, 2/sqrt(w))
    # Calculate weight using cumulative distribution function
    return 1 - 2 * cdf(norm_dist, -(w-l)/2)
end

function generate_weight_map()
    weight_map = Dict{Tuple{Int,Int}, Float64}()
    for w in 1:9  # Maximum games in a match is typically 9
        for l in 0:(w-1)
            weight_map[(w, l)] = calculate_weight(w, l)
        end
    end
    return weight_map
end

# Generate the weight map at module load time
const WEIGHT_MAP = generate_weight_map()

function result_to_weight(result::String)
    # Split result string and convert to integers
    scores = split(result, ':') .|> x -> parse(Int, x)
    x, y = scores
    
    # Return 0 for draws
    x == y && return 0.0
    
    # Determine winner and loser game counts
    winner_games = max(x, y)
    loser_games = min(x, y)
    
    # Calculate sign based on which player won
    sign_multiplier = x > y ? 1.0 : -1.0
    
    # Look up pre-calculated weight and apply sign
    return WEIGHT_MAP[(winner_games, loser_games)] * sign_multiplier
end

result_to_weight (generic function with 1 method)

#### 2.3.2 Parse the data in ITTF format

In [18]:
function convert_ittf_match(raw_match)
    # Extract player IDs
    player_a_id = raw_match["vw_matches___player_a_id_raw"]
    player_x_id = raw_match["vw_matches___player_x_id_raw"]

    # Extract player Assocs
    player_a_assoc = raw_match["vw_matches___assoc_a_raw"]
    player_x_assoc = raw_match["vw_matches___assoc_x_raw"]
    
    # Skip if not single match (player_b or player_y exists)
    if haskey(raw_match, "vw_matches___player_b_id_raw") && 
        !isnothing(raw_match["vw_matches___player_b_id_raw"]) ||
        haskey(raw_match, "vw_matches___player_y_id_raw") && 
        !isnothing(raw_match["vw_matches___player_y_id_raw"])
        return nothing
    end
    
    # Check null values (withdraw)
    if isnothing(player_a_id) || isnothing(player_x_id)
        return nothing
    end
    
    # Convert result from "A - X" to "A:X" format
    result = replace(raw_match["vw_matches___res_raw"], " - " => ":")
        
    # Get games result, remove possible "0:0" or space suffix
    games = strip(replace(raw_match["vw_matches___games_raw"], r"\s*0:0\s*$" => ""))

    # Compute the weight by result
    weight = result_to_weight(result)

    return Match(player_a_id, player_x_id, player_a_assoc, player_x_assoc, weight, result, games)
end

function process_matches(new_matches_json::String, event::Event)    
    data = JSON.parse(new_matches_json)
    
    new_match_count = 0

    # ITTF format has array of arrays
    for match_list in data
        for raw_match in match_list
            match = convert_ittf_match(raw_match)
            # Only add valid single matches
            if !isnothing(match)
                push!(event.match, match)
                new_match_count += 1
            end
        end
    end

    return new_match_count
end

process_matches (generic function with 1 method)

#### 2.3.3 Download the match data from the ITTF website

In [ ]:
function download_matches_from_ittf(events::Vector{Event})   
    match_count = 0

    # Create matches directory if it doesn't exist
    matches_dir = "matches"
    if !isdir(matches_dir)
        mkpath(matches_dir)
    end

    # Find the next available file number
    next_file_num = 0
    while isfile(joinpath(matches_dir, "matches_$(next_file_num).json"))
        next_file_num += 1
    end
    
    # Download matches from ITTF API
    for event in events
        event_id = event.id
        offset = 0
        all_new = true

        while all_new

            url = "https://www.results.ittf.link/index.php?option=com_fabrik&view=list&listid=31&Itemid=250&resetfilters=1&format=json&vw_matches___tournament_id_raw[value][]=$(event_id)&limit31=100&limitstart31=$(offset)"
            success = false
            
            while !success
                try
                    response = HTTP.get(url, cookies=COOKIES)
                    
                    if response.status != 200
                        println("Error: API request failed with status code: $(response.status)")
                        sleep(10)
                        continue
                    end
                    
                    # Process the downloaded JSON file
                    new_match_json = String(response.body)
                    new_match_count = process_matches(new_match_json, event)
                    
                    # Check if we have any new events
                    if new_match_count == 0
                        all_new = false
                        println("No more event data available")
                        break
                    else
                        all_new = new_match_count == 100
                        match_count += new_match_count
                    end

                    # Save to file
                    target_file = joinpath(matches_dir, "matches_$(next_file_num).json")
                    open(target_file, "w") do io
                        write(io, new_match_json)
                    end
                    println("Save matches to $(target_file)")
                    next_file_num += 1
                    
                    success = true
                catch e
                    println("Request failed: $(e)")
                    sleep(10)
                end
            end
            
            # Move to next batch
            offset += 100
            
            # Rate limiting
            sleep(0.5)

        end
    end
    
    return match_count
end

new_match_count = download_matches_from_ittf(new_events)
println("Downloaded $(new_match_count) matches")

### 2.4 Update players

Convert the ITTF format data into the Player structure.

#### 2.4.1 Detect new players

In [ ]:
function extract_player_ids(events::Vector{Event})
    player_ids = Set{Int}()
    
    for event in events
        for match in event.match
            if !haskey(players, match.player_a_id)
                push!(player_ids, match.player_a_id)
            end
            if !haskey(players, match.player_x_id)
                push!(player_ids, match.player_x_id)
            end
        end
    end
    
    return player_ids
end

new_player_ids = extract_player_ids(new_events)
println("There are $(length(new_player_ids)) new players")

#### 2.4.2 Parse the data in ITTF format

In [19]:
function parse_player_data(new_player_json::String)
    # Parse the JSON string
    data = JSON.parse(new_player_json)

    # Get player basic information
    # Get player profile information
    local profile
    try
        profile = data[1][1]["vw_profiles___profile_raw"]
    catch e
        return nothing
    end
    
    # Parse name and association (remove ID and association)
    name_with_id = data[1][1]["vw_profiles___name_raw"]
    name_match = match(r"^(.*?)\s*\(.*?\)$", name_with_id)
    name = name_match === nothing ? name_with_id : name_match.captures[1]
    
    # Parse sex
    sex = data[1][1]["vw_profiles___gender_raw"]

    # Parse association information
    assoc = nothing
    # Extract association from profile
    assoc_match = match(r"<img src='.*?/flags/([A-Z]{3})\.png'", profile)
    if assoc_match !== nothing
        assoc = assoc_match.captures[1]
    end
    
    # If the above method fails, try to extract from player_id
    if assoc === nothing
        # Match the content in the last parentheses
        assoc_match = match(r"\(([^()]*)\)[^()]*$", data[1][1]["vw_profiles___player_id"])
        if assoc_match !== nothing
            assoc = assoc_match.captures[1]
        end
    end
    
    # Use regular expressions to parse other fields
    yob_match = match(r"YoB:\s*(\d+)", profile)
    yob = yob_match === nothing ? nothing : parse(Int, yob_match.captures[1])

    # If yob is not found, compute it with age
    if yob === nothing
        age_match = match(r"Age:\s*(\d+)", profile)
        if age_match !== nothing
            age = parse(Int, age_match.captures[1])
            current_year = year(today())
            yob = current_year - age
        end
    end
    
    # Match handedness, could be Left-Hand, Right-Hand or Unknown Handness
    hand_match = match(r"Style:\s*(.*?Hand\w*)", profile)
    hand = hand_match === nothing ? nothing : hand_match.captures[1]
    
    # Match playing style, in the Style tag, after handedness and before parentheses
    style_match = match(r"Style:\s*.*?Hand\w*\s+(.*?)\s*\(", profile)
    style = style_match === nothing ? nothing : strip(style_match.captures[1])
    
    # Match grip style, in the Style tag, inside parentheses
    grip_match = match(r"Style:.*?\((.*?)\)", profile)
    grip = grip_match === nothing ? nothing : strip(grip_match.captures[1])
    
    # Create player history record, add current association to today's date by default
    history = Dict{String, Date}()
    if assoc !== nothing
        history[assoc] = today()
    end
    
    # Get player ID from the data
    player_id = data[1][1]["vw_profiles___player_id_raw"]
    
    # Create player data object
    player_data = Player(
        player_id,
        name,
        sex === nothing ? "U" : sex,
        history,  # History record containing default association information
        yob === nothing ? 0 : yob,
        hand === nothing ? "Unknown Handness" : hand,
        style === nothing ? "Unknown Style" : style,
        grip === nothing ? "Unknown Grip" : grip
    )
    
    return player_data
end

parse_player_data (generic function with 1 method)

#### 2.4.3 Download the player data from the ITTF website

In [15]:
function download_players_from_ittf(player_ids::Set{Int})
    # Ensure players directory exists
    isdir("players") || mkdir("players")
    
    players = Vector{Player}()

    # Initialize counters
    success_count = 0
    failure_count = 0
    
    for player_id in player_ids
        # Skip if player data already exists
        # if isfile("players/$(player_id).json")
        #     success_count += 1
        #     continue
        # end
        
        # Construct URL
        url = "https://www.results.ittf.link/index.php?option=com_fabrik&view=list&listid=33&resetfilters=1&format=json&limit33=100&limitstart27=0&vw_profiles___player_id_raw[value]=$(player_id)"
        
        # Make request with retry logic
        while true
            try
                response = HTTP.get(url, cookies=COOKIES)
                
                if response.status == 200
                    # Parse and save JSON data
                    player_json = String(response.body)
                    player = parse_player_data(player_json)
                    if player === nothing
                        println("Invalid player $(player_id)")
                        break                        
                    end
                    push!(players, player)

                    open("players/$(player_id).json", "w") do io
                        write(io, player_json)
                    end
                    
                    success_count += 1
                    println("Downloaded player: $(player.name)")
                    
                    break  # Success, exit retry loop
                else
                    failure_count += 1
                    println("Failed to fetch player #$(player_id), status: $(response.status)")
                end
                
            catch e
                failure_count += 1
                println("Error when fetching player #$(player_id): $e")
                sleep(10)  # Wait 10 seconds before retry
            end
        end
        
        sleep(0.1)  # Rate limiting between requests
    end
    
    println("Failed to fetch $(failure_count) players")
    return players
end

new_players = download_players_from_ittf(new_player_ids)
println("Downloaded $(length(new_players)) players")

Invalid player 108324
Invalid player 112003
Invalid player 111987
Invalid player 106938
Invalid player 109081
Invalid player 100831
Invalid player 106367
Invalid player 102155
Invalid player 115151
Invalid player 132569
Invalid player 112299
Invalid player 103782
Invalid player 100041
Invalid player 106241
Invalid player 106822
Invalid player 109950
Invalid player 107652
Invalid player 103761
Invalid player 101575
Invalid player 102522
Invalid player 102516
Invalid player 106638
Invalid player 105547
Invalid player 100450
Invalid player 108333
Invalid player 103484
Invalid player 100241
Invalid player 108663
Invalid player 100045
Invalid player 115262
Invalid player 106474
Invalid player 113425
Invalid player 112061
Invalid player 101460
Invalid player 104459
Invalid player 102477
Invalid player 105737
Invalid player 100036
Invalid player 104352
Invalid player 109393
Invalid player 105735
Invalid player 111951
Invalid player 104945
Invalid player 107040
Invalid player 103353
Invalid pl

## Step 3 - Convert the formats

In this step, we convert the data to my format and save it in `data` directory.

### 3.1 Update the players

#### 3.1.1 Merge the new players to existing ones

In [20]:
# Merge newly downloaded players into existing player data
function merge_players(existing_players::Dict{Int, Player}, new_players::Vector{Player})
    
    # Add newly downloaded players
    for player in new_players
        if haskey(existing_players, player.id)
            println("Error: existing player id: $(player.id)")
        else
            existing_players[player.id] = player
        end
    end
    
end

# Merge newly downloaded players into existing player data
merge_players(players, new_players)
println("Total players: $(length(players))")

Total players: 29079


#### 3.1.2 Update association information

Some players may change their association (e.g. ZHU Yuling), we should analyze the match data to check if the change occurs.

In [21]:
function update_player_associations(events::Vector{Event}, players::Dict{Int, Player})
    
    # Map deprecated association codes to correct ones
    assoc_mapping = Dict(
        "BLZ" => "BIZ",
        "ROM" => "ROU",
        "SIN" => "SGP",
        "SRL" => "SRI"
    )
    
    # Function to process association information for a single player
    function process_player_assoc(player_id, assoc, event_date)
        if !haskey(players, player_id) || assoc === nothing
            return
        end
        
        # Skip AIN association
        if assoc == "AIN"
            return
        end
        
        # Handle association code mapping
        if haskey(assoc_mapping, assoc)
            assoc = assoc_mapping[assoc]
        end
        
        player = players[player_id]
        # Only update if the association doesn't exist or the new date is later
        if !haskey(player.history, assoc) || event_date > player.history[assoc]
            player.history[assoc] = event_date
        end
    end
    
    # Iterate through all events and matches
    for event in events
        event_date = event.time
        for match in event.match
            process_player_assoc(match.player_a_id, match.player_a_assoc, event_date)
            process_player_assoc(match.player_x_id, match.player_x_assoc, event_date)
        end
    end
    
end

# Update players' association history
update_player_associations(new_events, players)
println("The associations are updated.")

update_player_associations (generic function with 1 method)

#### 3.1.3 Save the players to JSON

In [24]:
# Save player data to JSON file
function save_players_to_json(players, filename)
    # Create directory if it doesn't exist
    dir_path = dirname(filename)
    if !isdir(dir_path)
        mkpath(dir_path)
    end
    
    # Convert player data to serializable format
    serializable_players = []
    for (id, player) in players
        player_dict = Dict(
            "id" => player.id,
            "name" => player.name,
            "sex" => player.sex,
            "history" => Dict(assoc => string(date) for (assoc, date) in player.history),
            "yob" => player.yob,
            "hand" => player.hand,
            "style" => player.style,
            "grip" => player.grip
        )
        push!(serializable_players, player_dict)
    end
    
    # Write to JSON file
    open(filename, "w") do io
        JSON.print(io, serializable_players)
    end
    
    println("Saved data for $(length(players)) players to $filename")
end

# Save player data
save_players_to_json(players, "data/players.json")

Saved data for 29079 players to data/players.json


### 3.2 Update events and matches

#### 3.2.1 Merge the new events to the existing ones

In [ ]:
# Merge new events into existing events and return modified dates
function merge_events(events::Dict{Date, Vector{Event}}, new_events::Vector{Event})
    # Track modified dates
    modified_dates = Set{Date}()
    
    # Merge new events
    for new_event in new_events
        date = new_event.time
        
        # Record this date as modified
        push!(modified_dates, date)
        
        # If this date doesn't exist yet, create a new entry
        if !haskey(events, date)
            events[date] = Event[]
        end
        
        # Check if an event with the same ID already exists
        existing_index = findfirst(e -> e.id == new_event.id, events[date])
        
        if existing_index !== nothing
            # Update existing event
            events[date][existing_index] = new_event
        else
            # Add new event
            push!(events[date], new_event)
        end
    end
    
    return modified_dates
end

modified_dates = merge_events(events, new_events)
println("Number of modified dates: $(length(modified_dates))")
println("Total number of events organized by date: $(length(events))")

#### 3.2.2 Save the events/matches to JSON

In [27]:
function save_events_to_files(events_data::Dict{Date, Vector{Event}}, modified_dates::Set{Date})
    # Create necessary directories if they don't exist
    mkpath("data")
    mkpath("data/events")

    # Only save files for dates in modified_dates
    saved_count = 0
    for date in modified_dates
        if haskey(events_data, date)
            date_events = events_data[date]
            
            # Convert Event structs to dictionaries
            events_dict = []
            for event in date_events
                event_dict = Dict(
                    "id" => event.id,
                    "name" => event.name,
                    "weight" => event.weight,
                    "time" => Dates.format(event.time, "yyyymmdd"),
                    "match" => [
                        Dict(
                            "player_a_id" => m.player_a_id,
                            "player_x_id" => m.player_x_id,
                            "player_a_assoc" => m.player_a_assoc,
                            "player_x_assoc" => m.player_x_assoc,
                            "weight" => m.weight,
                            "result" => m.result,
                            "games" => m.games
                        ) for m in event.match
                    ]
                )
                push!(events_dict, event_dict)
            end

            # Write to JSON file with date as filename
            open("data/events/$date.json", "w") do f
                JSON.print(f, events_dict, 2)
            end
            
            saved_count += 1
        end
    end
    
    println("Successfully saved $(saved_count) modified date files to data/events/")
end

# save_events_to_files(events, modified_dates)

save_events_to_files (generic function with 1 method)

In [24]:
# 将事件数据从 event_id => Event 格式转换为 date => Vector{Event} 格式
function convert_events_format(events_dict::Dict{Int64, Event})
    # 创建新的数据结构：日期 => 事件列表
    date_events = Dict{Date, Vector{Event}}()
    
    # 遍历所有事件
    for (_, event) in events_dict
        event_date = event.time
        
        # 如果该日期尚未在字典中，创建一个新的空数组
        if !haskey(date_events, event_date)
            date_events[event_date] = Event[]
        end
        
        # 将事件添加到对应日期的数组中
        push!(date_events[event_date], event)
    end
    
    return date_events
end

# 示例用法
events_by_date = convert_events_format(events_dict)
println("共有 $(length(keys(events_by_date))) 个不同的日期")


In [25]:
events_by_date

Dict{Date, Vector{Event}} with 1290 entries:
  Date("2012-05-20") => [Event(90, "World Tour, Korea Open, Incheon (KOR)", 1.2…
  Date("2010-03-21") => [Event(53, "Pro Tour German Open, Berlin (GER)", 1.2, D…
  Date("2022-02-27") => [Event(2563, "Europe Top-16 Cup, Montreux (SUI)", 1.5, …
  Date("2008-03-16") => [Event(785, "Pro Tour Kuwait Open, Kuwait City (KUW)", …
  Date("2012-11-25") => [Event(1965, "China vs. World Team Challenge, Shanghai …
  Date("2010-04-08") => [Event(622, "Caribbean Junior and Cadet Championships, …
  Date("2015-01-11") => [Event(205, "Team World Cup, Dubai (UAE)", 2.0, Date("2…
  Date("2022-09-25") => [Event(2544, "WTT Youth Contender, Tbilisi (GEO)", 0.3,…
  Date("2011-09-30") => [Event(709, "Asian Cup, Changsha (CHN)", 1.5, Date("201…
  Date("2022-03-17") => [Event(2534, "WTT Feeder, Doha (QAT)", 1.0, Date("2022-…
  Date("2024-01-21") => [Event(2896, "Pan American Cup, Corpus Christi (USA)", …
  Date("2017-10-08") => [Event(503, "ITTF Challenge Polish Open,

In [28]:
# 保存更新后的事件数据
function save_updated_events(updated_events)
    # 获取所有日期
    all_dates = Set{Date}(keys(updated_events))
    
    # 调用已有的保存函数，传入所有日期
    save_events_to_files(updated_events, all_dates)
    
    println("已成功保存所有 $(length(all_dates)) 个日期的事件数据")
end

# 示例用法
save_updated_events(events_by_date)


## Step 4 - Run the ranking algorithm

I develop a dedicated algorithm for the ITTF/WTT rankings. It is based on the classic ELO algorithm, but I make some difference. Here I will make some explanation.

Table tennis has its uniqueness compared to some other typical sports with rating, e.g. tennis and chess. One of the main chellenges is that WTT events lack sufficient influence, thus some players do not regard WTT events as their most important professional activities. They can participate in high-level domestic professional events in China, Japan, and some other countries and regions. Some players even do not participate in any open events and can reach or maintain a considerably high competitive level just through in-team training. For example, FAN Siqi only participate in one WTT event (China Smash) in 2024, but in this event she defeat many high-level players, inclusing WANG Yidi, CHENG I-Ching, and MORI Sakura. Another challenge is that top table tennis players have extreme dominance. They may participate in a large number of events during their peak periods, with a winning rate exceeding 90%. This leads to a situation that their ratings may be quite high. When their competitive level declines due to aging or injury, the number of events they participate in will significantly decrease, resulting in insufficient rating deductions to reach an appropriate level. Of course, the current rating system of WTT can overcome the latter challenge with cleaning points after one year, and it would be better if WTT could enhance its influence and encourage high-level players to participate in more WTT events. Before that, please allow me to still establish a model based on the ELO method.

My algorithm makes two modifications to the clasic ELO algorithm. Both of them are implemented with a sigmoid-like function.

**The first point is "long jump".** In the classic algorithm, a player with a lower rating (e.g. HARIMOTO Tomokazu) defeating a player with a much higher rating (e.g. MA Long) will only increase a limited number of points. For a long time, everyone knew that the talented young player had extraordinary competitive level, but due to the sublinear rating growth his ranking would remain lower than the position recognized by the public. A even more obvious example comes from KIM Kum Yong, who defeated SUN Yingsha, WANG Yidi, and HARIMOTO Miwa at the Asian Championships, but still had a low ranking under the WTT rating system because she participated in no WTT events. To allow a talented player who is new to WTT events to quickly converge their rating to the appropriate level, I enable their points to increase rapidly, and the magnitude of this increase depends on the current rating of his/her oppponent. The "long jump" occasionally causes some players (e.g. TANAKA Yuta) to have inflated points. However, since situations like KIM Kum Yong occur more frequently, the benefits of "long jump" can outweigh the drawbacks.

**The second point is "centripetal force".** In simple terms, a player with a higher rating will find it more difficult to increase their points and easier to lose points, especially when losing to an opponent with a much lower rating than themselves. This is to address the issue of excessive inflation of points for top players, allowing their points to quickly "fall to earth" when their performance declines. On the other hand, the "long jump" results in a positive sum of point changes for the two players in a match, requiring a negative sum factor to make a balance.

This algorithm has an accuracy rate of around **76%** in predicting the results of competitions after 2018/01/01, while WTT's current point rules have an accuracy rate of only about 66%.

*Pay attention:* High predictive accuracy is not the sole criterion to a ranking system. I have some algorithms that perform better in this metric, but employing them would result in drastic ranking fluctuations, rendering the rankings at a certain time point ineffective as a reliable reference. So, such algorithms cannot be deemed valid ranking algorithms.

### 4.1 Algorithm to update rating

Inputs:
- $r_1$: rating of player 1 before the match.
- $r_2$: rating of player 2 before the match.
- $w_m$: weight of the match.
- $w_e$: weight of the event.

Outputs:
- $r'_1$: rating of player 1 after the match.
- $r'_2$: rating of player 2 after the match.

Algorithm (take $r_1\to r'_1$ as an example):

1. The ELO expectation $e_1 = \frac{1}{1 + 10^{\frac{r_1 - r_2}{D_1}}}$.
2. Raw rating delta $\Delta_1 = w_m \times w_e \times W \times (a_1 - e_1)$, where $a_1$ is 1 when player 1 wins and 0 when player 1 loses.
3. Centripetal force
    - If $a_1 = 0$, $\Delta'_1 = \Delta_1 \times \frac{2}{1 + 10^{-\frac{r_1 - r_0}{D_2}}}$, where $r_0$ is the default rating of a player.
    - If $a_1 = 1$, $\Delta'_1 = \Delta_1 \times \frac{2}{1 + 10^{\frac{r_1 - r_c}{D_2}}}$, where $r_c$ is a ceil rating of a player.
4. Centripetal force (by opponent)
    - If $a_1 = 0$, $\Delta''_1 = \Delta'_1 \times \frac{2}{1 + 10^{-\frac{r_1 - r_2}{D_3}}}$.
    - If $a_1 = 1$, $\Delta''_1 = \Delta'_1 \times \frac{2}{1 + 10^{\frac{r_1 - r_2}{D_3}}}$.
5. Result of centripetal force $r'_1 = r_1 + \Delta''_1$
6. Long jump (no "long fall" in my implementation, i.e. $\beta=0$)
    - If $r_1 < r'_1 < r_2$, $r'_1 = r'_1 + \alpha \times (r_2 - r'_1)$.
    - If $r_1 > r'_1 > r_2$, $r'_1 = r'_1 + \beta \times (r_2 - r'_1)$.

Parameters (fine-tuned using the ITTF/WTT data):

|  $W$  |  $D_1$  |  $D_2$  |  $D_3$  |  $r_0$  |  $r_c$  |  $\alpha$  |  $\beta$  |
| ----- | ------- | ------- | ------- | ------- | ------- | ---------- | --------- |
|   50  |  1000   |  1000   |   250   |  1500   |  3000   |    0.25    |    0    |



In [9]:
# Parameters
const W = 50.0
const D1 = 1000.0
const D2 = 1000.0
const D3 = 250.0
const r0 = 1500.0
const rc = 3000.0
const α = 0.25
const β = 0

const date_0 = Date("2004-01-01")
const date_1 = Date("2010-01-01")

function update_rating(r1::Float64, r2::Float64, wm::Float32, we::Float32, date::Date)
    a1 = wm > 0.0 ? 1.0 : (wm < 0.0 ? 0.0 : 0.5)
    wm = abs(wm)
    if date < date_0
        wm *= 2
    end
    if date < date_1
        wm *= 2
    end
    
    # Calculate opponent's result
    a2 = 1.0 - a1
    
    # Step 1: Calculate ELO expectation
    e1 = 1.0 / (1.0 + 10.0^((r1 - r2) / D1))
    e2 = 1.0 / (1.0 + 10.0^((r2 - r1) / D1))
    
    # Step 2: Calculate raw rating delta
    Δ1 = wm * we * W * (a1 - e1)
    Δ2 = wm * we * W * (a2 - e2)
    
    # Step 3: Centripetal force (based on default and ceiling ratings)
    if Δ1 < 0
        Δ1_prime = Δ1 * (2.0 / (1.0 + 10.0^(-(r1 - r0) / D2)))
    else
        Δ1_prime = Δ1 * (2.0 / (1.0 + 10.0^((r1 - rc) / D2)))
    end
    
    if Δ2 < 0
        Δ2_prime = Δ2 * (2.0 / (1.0 + 10.0^(-(r2 - r0) / D2)))
    else
        Δ2_prime = Δ2 * (2.0 / (1.0 + 10.0^((r2 - rc) / D2)))
    end
    
    # Step 4: Centripetal force (based on opponent's rating)
    if Δ1 < 0
        Δ1_dprime = Δ1_prime * (2.0 / (1.0 + 10.0^(-(r1 - r2) / D3)))
    else
        Δ1_dprime = Δ1_prime * (2.0 / (1.0 + 10.0^((r1 - r2) / D3)))
    end
    
    if Δ2 < 0
        Δ2_dprime = Δ2_prime * (2.0 / (1.0 + 10.0^(-(r2 - r1) / D3)))
    else
        Δ2_dprime = Δ2_prime * (2.0 / (1.0 + 10.0^((r2 - r1) / D3)))
    end
    
    # Step 5: Result of centripetal force
    r1_prime = r1 + Δ1_dprime
    r2_prime = r2 + Δ2_dprime
    
    # Step 6: Long jump
    if r1 < r1_prime && r1_prime < r2
        r1_prime = r1_prime + α * (r2 - r1_prime)
    elseif r1 > r1_prime && r1_prime > r2
        r1_prime = r1_prime + β * (r2 - r1_prime)
    end
    
    if r2 < r2_prime && r2_prime < r1
        r2_prime = r2_prime + α * (r1 - r2_prime)
    elseif r2 > r2_prime && r2_prime > r1
        r2_prime = r2_prime + β * (r1 - r2_prime)
    end
    
    return r1_prime, r2_prime
end

update_rating (generic function with 1 method)

#### 4.2 Compute the active period of each player

We calculate the active period of each player. If a player has not participated in any ITTF/WTT events for at least 1,000 days, that period is deemed inactive. Inactive players will not appear in the output rankings.

In [10]:
struct Period
    start::Date
    fin::Date
end

function compute_active_periods(events::Dict{Date, Vector{Event}})
    # Initialize dictionary to store active periods for each player
    active_periods = Dict{Int, Vector{Period}}()
    
    # Track the last match date for each player
    last_match_date = Dict{Int, Date}()
    
    # Process all events in chronological order
    sorted_dates = sort(collect(keys(events)))
    
    for date in sorted_dates
        for event in events[date]
            for match in event.match
                for player_id in [match.player_a_id, match.player_x_id]
                    if haskey(players, player_id)
                        update_player_period(player_id, date, active_periods, last_match_date)
                    end
                end
            end
        end
    end
    
    # Close any open active periods
    current_date = Dates.today()
    for (player_id, last_date) in last_match_date
        if haskey(active_periods, player_id) && length(active_periods[player_id]) > 0
            last_period = active_periods[player_id][end]
            if last_period.fin == Date(9999, 12, 31)  # If period is still open
                if Dates.value(current_date - last_date) <= 1000
                    active_periods[player_id][end] = Period(last_period.start, current_date)
                else
                    active_periods[player_id][end] = Period(last_period.start, last_date)
                end
            end
        end
    end
    
    return active_periods
end

function update_player_period(player_id::Int, date::Date, 
                             active_periods::Dict{Int, Vector{Period}}, 
                             last_match_date::Dict{Int, Date})
    # Initialize active period record for new players
    if !haskey(active_periods, player_id)
        active_periods[player_id] = [Period(date, Date(9999, 12, 31))]
        last_match_date[player_id] = date
        return
    end
    
    # Get player's last match date
    last_date = last_match_date[player_id]
    
    # If more than 1000 days have passed, start a new active period
    if Dates.value(date - last_date) > 1000
        # Close the previous active period
        if length(active_periods[player_id]) > 0 && active_periods[player_id][end].fin == Date(9999, 12, 31)
            active_periods[player_id][end] = Period(active_periods[player_id][end].start, last_date)
        end
        
        # Start a new active period
        push!(active_periods[player_id], Period(date, Date(9999, 12, 31)))
    end
    
    # Update the last match date
    last_match_date[player_id] = date
end

active_periods = compute_active_periods(events)
println("Calculated $(length(active_periods)) players' active periods")

Calculated 28969 players' active periods


#### 4.3 Output ranking data by Typst

We use Typst to generate ranking data. Each output file has four pages, and there are 32 players in each page, i.e., each output file provides a top-200 ranking list.

In [14]:
function save_rankings(date::Date, type::String, rating::Dict{Int, Float64}, last_ranking::Dict{Int, Int}, filename::String="")
    # Create directory structure
    year_dir = "history/$(year(date))"
    if !isdir(year_dir)
        mkpath(year_dir)
    end
    
    # Create filename with 2-digit month format
    month_str = lpad(month(date), 2, '0')
    output_filename = !isempty(filename) ? filename : "$(year_dir)/$(type)S-$(month_str).typ"
    
    # Filter active players
    active_players = Int[]
    for (player_id, periods) in active_periods
        # Check if player is active on the current date
        is_active = false
        for period in periods
            if period.start <= date <= period.fin
                is_active = true
                break
            end
        end
        
        if is_active && haskey(rating, player_id) && players[player_id].sex == type
            push!(active_players, player_id)
        end
    end
    
    # Sort by rating and take top 200
    sort!(active_players, by=id -> -rating[id])
    
    # Create current rankings dictionary
    current_ranking = Dict{Int, Int}()
    for (rank, player_id) in enumerate(active_players)
        current_ranking[player_id] = rank
    end
    
    # Output top-200 players
    if length(active_players) > 200
        active_players = active_players[1:200]
    end
    
    # Write to file
    open(output_filename, "w") do f
        # Write template information
        if isempty(filename)
            write(f, "#import \"../../template.typ\": *\n")
        else
            write(f, "#import \"template.typ\": *\n")
        end
        write(f, "#set text(font: (\"Microsoft YaHei\"))\n\n")
        
        # Determine title
        title = type == "M" ? "Men's Singles" : "Women's Singles"
        
        # 25 players per page, 8 pages total
        for page in 1:8
            start_rank = (page - 1) * 25 + 1
            end_rank = min(page * 25, length(active_players))
            
            # Write table header
            write(f, "#figure(\n")
            write(f, "  caption: \"$(title) ($(start_rank) - $(end_rank))\",\n")
            write(f, "    table(\n")
            write(f, "      columns: 9,\n")
            write(f, "      [\\#], [Player],[Age], [Assoc.],  [Hand], [Grip], [Style], [Rating], [\$Delta\$],\n")
            
            # Write player information
            for i in start_rank:end_rank
                if i <= length(active_players)
                    player_id = active_players[i]
                    player = players[player_id]
                    
                    # Calculate age
                    age = year(date) - player.yob
                    
                    # Get association info
                    assoc = "?"
                    latest_date = today() + Day(1)
                    for (association, change_date) in player.history
                        if date <= change_date < latest_date
                            assoc = association
                            latest_date = change_date
                        end
                    end
                    
                    # Get playing hand
                    hand = player.hand == "Right-Hand" ? "#right" : 
                           player.hand == "Left-Hand" ? "#left" : "?"
                    
                    # Get grip style
                    grip = player.grip == "ShakeHand" ? "#shakehand" : 
                           player.grip == "Penhold" ? "#penhold" : "?"
                    
                    # Get playing style
                    style = player.style == "Attack" ? "#attack" : 
                            player.style == "Defence" ? "#defense" : "?"
                    
                    # Get ranking change
                    delta = haskey(last_ranking, player_id) ? 
                            "#delta($(last_ranking[player_id] - i))" : "NEW"
                    
                    # Write player row
                    write(f, "      [$(i)], [#name(\"$(player.name)\")], [#age($(age))], [#assoc(\"$(assoc)\")], ")
                    write(f, "[$(hand)], [$(grip)], [$(style)], [*$(floor(Int, rating[player_id]))*], [$(delta)],\n")
                end
            end
            
            # End table
            write(f, "    )\n")
            write(f, "  )\n")
            
            # Add page break except for the last page
            if page < 8
                write(f, "#pagebreak()\n\n")
            end
        end
    end
    
    println("Rankings saved to file: $(output_filename)")
    
    # Return the current rankings
    return current_ranking
end

save_rankings (generic function with 2 methods)

#### 4.4 Run the ranking algorithm

In [18]:
function compute_rankings()
    # 初始化玩家评分字典
    ratings = Dict{Int, Float64}()
    
    # 初始化上次排名字典
    last_ranking = Dict{Int, Int}()

    # 记录历史最高分
    highest_ratings = Dict{Int, Float64}()
    
    # 设置开始日期和结束日期
    start_date = Date("2004-01-01")
    end_date = today()
    
    # 获取所有事件并按日期排序
    all_events = Event[]
    for (date, date_events) in events
        for event in date_events
            push!(all_events, event)
        end
    end
    
    # 按日期排序
    sort!(all_events, by = e -> e.time)
    
    # 当前月份
    current_month = Month(0)
    
    # 遍历每个事件
    for event in all_events
        # 检查是否需要保存排名（每月一次）
        if event.time >= start_date
            ranking_m = save_rankings(start_date, "M", ratings, last_ranking)
            ranking_f = save_rankings(start_date, "W", ratings, last_ranking)
            # 清空并重新创建last_ranking
            empty!(last_ranking)
            
            # 合并男女排名为新的last_ranking
            for (player, rank) in ranking_m
                last_ranking[player] = rank
            end
            
            for (player, rank) in ranking_f
                last_ranking[player] = rank
            end
            
            # 更新start_date到下个月的第一天
            start_date = start_date + Month(1)

            # 更新历史最高分
            for (id, rating) in ratings
                if ratings[id] > highest_ratings[id]
                    highest_ratings[id] = ratings[id]
                end
            end
        end

        # 处理事件中的每场比赛
        for match in event.match
            # 获取玩家ID
            player1_id = match.player_a_id
            player2_id = match.player_x_id
            
            # 如果是新玩家，初始化评分
            if !haskey(ratings, player1_id)
                ratings[player1_id] = r0
                highest_ratings[player1_id] = r0
            end
            
            if !haskey(ratings, player2_id)
                ratings[player2_id] = r0
                highest_ratings[player2_id] = r0
            end
            
            # 更新评分
            r1, r2 = update_rating(ratings[player1_id], ratings[player2_id], match.weight, event.weight, event.time)
            ratings[player1_id] = r1
            ratings[player2_id] = r2
        end
    end

    save_rankings(end_date, "M", ratings, last_ranking, "MS-latest.typ")
    save_rankings(end_date, "W", ratings, last_ranking, "WS-latest.typ")

    # 返回按照最高分排序的字典
    sorted_highest_ratings = sort(collect(highest_ratings), by=x->x[2], rev=true)
    sorted_highest_ratings
end

compute_rankings (generic function with 1 method)

In [19]:
highest_ratings = compute_rankings()

Rankings saved to file: history/2004/MS-01.typ
Rankings saved to file: history/2004/WS-01.typ
Rankings saved to file: history/2004/MS-02.typ
Rankings saved to file: history/2004/WS-02.typ
Rankings saved to file: history/2004/MS-03.typ
Rankings saved to file: history/2004/WS-03.typ
Rankings saved to file: history/2004/MS-04.typ
Rankings saved to file: history/2004/WS-04.typ
Rankings saved to file: history/2004/MS-05.typ
Rankings saved to file: history/2004/WS-05.typ
Rankings saved to file: history/2004/MS-06.typ
Rankings saved to file: history/2004/WS-06.typ
Rankings saved to file: history/2004/MS-07.typ
Rankings saved to file: history/2004/WS-07.typ
Rankings saved to file: history/2004/MS-08.typ
Rankings saved to file: history/2004/WS-08.typ
Rankings saved to file: history/2004/MS-09.typ
Rankings saved to file: history/2004/WS-09.typ
Rankings saved to file: history/2004/MS-10.typ
Rankings saved to file: history/2004/WS-10.typ
Rankings saved to file: history/2004/MS-11.typ
Rankings save

29048-element Vector{Pair{Int64, Float64}}:
 110579 => 3232.984671554606
 105649 => 3205.4081294282087
 112019 => 3193.0421549499483
 131163 => 3184.4411677811245
 105648 => 3183.043484589099
 109961 => 3179.4442097927113
 121404 => 3164.2423072512443
 121411 => 3149.397248368951
 105350 => 3133.892268913907
 102265 => 3129.958935571988
        ⋮
 115197 => 1500.0
 124376 => 1500.0
 202432 => 1500.0
 203185 => 1500.0
 207045 => 1500.0
 203994 => 1500.0
 105626 => 1500.0
 131277 => 1500.0
 206276 => 1500.0

In [20]:
for (id, value) in highest_ratings
    if value >= 2800
        println("$(players[id].name) \t => \t $value")
    end
end

ZHANG Yining 	 => 	 3232.984671554606
MA Long 	 => 	 3205.4081294282087
CHEN Meng 	 => 	 3193.0421549499483
SUN Yingsha 	 => 	 3184.4411677811245
MA Lin 	 => 	 3183.043484589099
WANG Hao 	 => 	 3179.4442097927113
FAN Zhendong 	 => 	 3164.2423072512443
WANG Manyu 	 => 	 3149.397248368951
LI Xiaoxia 	 => 	 3133.892268913907
DING Ning 	 => 	 3129.958935571988
WANG Chuqin 	 => 	 3127.396077819111
WANG Nan 	 => 	 3110.2574094885126
XU Xin 	 => 	 3098.7466817942627
WANG Liqin 	 => 	 3096.319919334347
LIU Shiwen 	 => 	 3094.37439676349
ZHU Yuling 	 => 	 3083.33458289673
ZHANG Jike 	 => 	 3075.116317347148
GUO Yue 	 => 	 3071.9917205649094
LIANG Jingkun 	 => 	 3064.454186448741
WANG Yidi 	 => 	 3053.005792250906
LIN Shidong 	 => 	 3047.1570005215967
SAMSONOV Vladimir 	 => 	 3031.547835797696
BOLL Timo 	 => 	 3019.031719602568
ITO Mima 	 => 	 3016.5401656453046
GUO Yan 	 => 	 3015.005527980969
HAO Shuai 	 => 	 3011.6313777949404
FENG Tianwei 	 => 	 3011.0775581573876
CHEN Xingtong 	 => 	 3003.8

#### 4.5 Translate to Chinese

In [17]:
function translate(src, dst)
    # 读取源文件内容
    text = read(src, String)
    
    # 定义翻译字典
    translation = Dict(
        "Player" => "运动员",
        "Assoc." => "协会",
        "Rating" => "积分",
        "template.typ" => "template_CN.typ",
        "Hand" => "手",
        "Grip" => "握拍",
        "Style" => "削球",
        "Age" => "年龄"
    )
    
    # 从翻译文件中读取更多翻译
    if isfile("translate.txt")
        open("translate.txt", "r") do f
            for line in eachline(f)
                words = split(line, ",")
                if length(words) >= 2
                    translation[strip(words[1])] = strip(words[2])
                end
            end
        end
    end
    
    # 替换文本中的单词
    for (eng, chn) in translation
        text = replace(text, eng * "]" => chn * "]")
        text = replace(text, eng * "\"" => chn * "\"")
    end
    
    # 写入目标文件
    write(dst, text)
end

# 创建中文历史文件夹
for year in 2004:2025
    dir_name = "history/$year"
    cn_dir_name = "history_CN/$year"
    
    if !isdir(cn_dir_name)
        mkdir(cn_dir_name)
    end
    
    # 翻译该年份下的所有文件
    if isdir(dir_name)
        for file_name in readdir(dir_name)
            file_path = "$dir_name/$file_name"
            cn_file_path = "$cn_dir_name/$file_name"
            translate(file_path, cn_file_path)
            println("已翻译 $file_path")
        end
    end
end

# 翻译最新排名文件
for event in ["MS", "WS"]
    translate("$event-latest.typ", "$event-latest_CN.typ")
end

println("翻译完成")


已翻译 history/2004/MS-01.typ
已翻译 history/2004/MS-02.typ
已翻译 history/2004/MS-03.typ
已翻译 history/2004/MS-04.typ
已翻译 history/2004/MS-05.typ
已翻译 history/2004/MS-06.typ
已翻译 history/2004/MS-07.typ
已翻译 history/2004/MS-08.typ
已翻译 history/2004/MS-09.typ
已翻译 history/2004/MS-10.typ
已翻译 history/2004/MS-11.typ
已翻译 history/2004/MS-12.typ
已翻译 history/2004/WS-01.typ
已翻译 history/2004/WS-02.typ
已翻译 history/2004/WS-03.typ
已翻译 history/2004/WS-04.typ
已翻译 history/2004/WS-05.typ
已翻译 history/2004/WS-06.typ
已翻译 history/2004/WS-07.typ
已翻译 history/2004/WS-08.typ
已翻译 history/2004/WS-09.typ
已翻译 history/2004/WS-10.typ
已翻译 history/2004/WS-11.typ
已翻译 history/2004/WS-12.typ
已翻译 history/2005/MS-01.typ
已翻译 history/2005/MS-02.typ
已翻译 history/2005/MS-03.typ
已翻译 history/2005/MS-04.typ
已翻译 history/2005/MS-05.typ
已翻译 history/2005/MS-06.typ
已翻译 history/2005/MS-07.typ
已翻译 history/2005/MS-08.typ
已翻译 history/2005/MS-09.typ
已翻译 history/2005/MS-10.typ
已翻译 history/2005/MS-11.typ
已翻译 history/2005/MS-12.typ
已翻译 history/2005/WS-01.typ
已

## Appendix

These code blocks are used to generate files in my format entirely from the files in ITTF format.

### A.1 Read player files

In [20]:
function read_all_players()
    players_dict = Dict{Int64, Player}()
    
    # 遍历players文件夹下的所有JSON文件
    for file in readdir("players")
        if endswith(file, ".json")
            try
                # 读取JSON文件内容
                player_json = read("players/$file", String)
                # 解析玩家数据
                player = parse_player_data(player_json)
                if player !== nothing
                # 将成功解析的Player添加到字典中
                    players_dict[player.id] = player
                end
            catch e
                println("处理文件 $file 时出错: $e")
            end
        end
    end
    
    println("成功加载 $(length(players_dict)) 名玩家数据")
    return players_dict
end

player_dict = read_all_players()

Dict{Int64, Player} with 29079 entries:
  122168 => Player(122168, "KHUDAIBERDIEV Zhasur", "M", Dict("KGZ"=>Date("2025-…
  113778 => Player(113778, "HERNANDEZ Natalia", "W", Dict("COL"=>Date("2025-04-…
  137206 => Player(137206, "MECKL Matiss", "M", Dict("ISL"=>Date("2025-04-13"))…
  201757 => Player(201757, "ZAMORA Gonzalo", "M", Dict("PER"=>Date("2025-04-13"…
  107905 => Player(107905, "RIVERA Justo", "M", Dict("PUR"=>Date("2025-04-13"))…
  103539 => Player(103539, "HELMY Ashraf", "M", Dict("EGY"=>Date("2025-04-13"))…
  137444 => Player(137444, "VIDAN Rino", "M", Dict("CRO"=>Date("2025-04-13")), …
  144721 => Player(144721, "CELIS Sara", "W", Dict("ESP"=>Date("2025-04-13")), …
  144690 => Player(144690, "LABANTI Precious", "M", Dict("GHA"=>Date("2025-04-1…
  100281 => Player(100281, "AL-HAMMADI Abdulla", "M", Dict("QAT"=>Date("2025-04…
  122266 => Player(122266, "KOH Jason", "M", Dict("BRU"=>Date("2025-04-13")), 2…
  102644 => Player(102644, "ASHRAF Ahmed", "M", Dict("EGY"=>Date("202

### A.2 Read events

In [21]:
function load_events_from_files()
    events_dir = "events"
    events_by_id = Dict{Int, Event}()
    empty_events = Dict{Date, Vector{Event}}()
    
    file_num = 0
    while isfile(joinpath(events_dir, "events_$(file_num).json"))
        file_path = joinpath(events_dir, "events_$(file_num).json")
        
        open(file_path, "r") do file
            json_data = read(file, String)
            events, _ = process_events(empty_events, json_data)
            
            for event in events
                events_by_id[event.id] = event
            end
        end
        
        file_num += 1
    end
    
    println("总共加载了 $(length(events_by_id)) 个赛事")
    return events_by_id
end

events_dict = load_events_from_files()

Dict{Int64, Event} with 1694 entries:
  2261 => Event(2261, "ITTF Challenge Polish Open, Gliwice (POL)", 1.0, Date("2…
  719  => Event(719, "ITTF Junior Circuit Czech Junior and Cadet Open, Hodonin …
  1703 => Event(1703, "JAPAN Open, Kobe (JPN)", 1.2, Date("2004-09-26"), Match[…
  1028 => Event(1028, "Northern European Youth Championships, Haapsalu (EST)", …
  699  => Event(699, "13th Ibero-American Youth Championships, Buenos Aires (AR…
  831  => Event(831, "ITTF World Junior Circuit Venezuela Junior Open, Valencia…
  1299 => Event(1299, "CHINA Open, Qingdao City (CHN)", 1.2, Date("2002-06-02")…
  2841 => Event(2841, "WTT Youth Star Contender, Doha (QAT)", 0.4, Date("2024-1…
  319  => Event(319, "ITTF North America Cup, Markham (CAN)", 0.75, Date("2015-…
  687  => Event(687, "Women's World Cup, Kuala Lumpur (MAS)", 2.0, Date("2010-0…
  1812 => Event(1812, "GERMAN Open, Bayreuth (GER)", 1.2, Date("2006-11-12"), M…
  2844 => Event(2844, "WTT Youth Contender, Rionegro (COL)", 0.3, Date(

In [22]:
function load_matches_into_events(events_dict)
    matches_dir = "matches"
    new_match_count = 0
    
    file_num = 0
    while isfile(joinpath(matches_dir, "matches_$(file_num).json"))
        file_path = joinpath(matches_dir, "matches_$(file_num).json")
        
        try
            open(file_path, "r") do file
                matches_data = JSON.parse(file)
            
                # ITTF格式是数组的数组
                for match_list in matches_data
                    for raw_match in match_list
                        event_id = raw_match["vw_matches___tournament_id_raw"]
                        
                        match = convert_ittf_match(raw_match)
                        
                        # 只添加有效的单打比赛
                        if !isnothing(match)
                            push!(events_dict[event_id].match, match)
                            new_match_count += 1
                        end
                    end
                end
            end
            println("已读取 $(file_path)，当前比赛数 $(new_match_count)")
            file_num += 1
        catch e
            println("处理比赛文件 matches_$(file_num).json 时出错: $e")
            file_num += 1
        end
    end
    
    println("成功加载了 $(new_match_count) 场比赛到赛事中")
    return events_dict
end

events_dict = load_matches_into_events(events_dict)


Dict{Int64, Event} with 1694 entries:
  2261 => Event(2261, "ITTF Challenge Polish Open, Gliwice (POL)", 1.0, Date("2…
  719  => Event(719, "ITTF Junior Circuit Czech Junior and Cadet Open, Hodonin …
  1703 => Event(1703, "JAPAN Open, Kobe (JPN)", 1.2, Date("2004-09-26"), Match[…
  1028 => Event(1028, "Northern European Youth Championships, Haapsalu (EST)", …
  699  => Event(699, "13th Ibero-American Youth Championships, Buenos Aires (AR…
  831  => Event(831, "ITTF World Junior Circuit Venezuela Junior Open, Valencia…
  1299 => Event(1299, "CHINA Open, Qingdao City (CHN)", 1.2, Date("2002-06-02")…
  2841 => Event(2841, "WTT Youth Star Contender, Doha (QAT)", 0.4, Date("2024-1…
  319  => Event(319, "ITTF North America Cup, Markham (CAN)", 0.75, Date("2015-…
  687  => Event(687, "Women's World Cup, Kuala Lumpur (MAS)", 2.0, Date("2010-0…
  1812 => Event(1812, "GERMAN Open, Bayreuth (GER)", 1.2, Date("2006-11-12"), M…
  2844 => Event(2844, "WTT Youth Contender, Rionegro (COL)", 0.3, Date(

In [23]:
events_dict[2711]

Event(2711, "ITTF World Youth Championships, Nova Gorica (SLO)", 0.8f0, Date("2023-12-03"), Match[Match(137237, 203067, "CHN", "CHN", 0.8663856f0, "4:1", "6:11 11:6 11:9 11:2 11:3"), Match(135049, 134011, "CHN", "ROU", 0.9544997f0, "4:0", "11:3 13:11 11:5 11:5"), Match(143604, 203029, "JPN", "CHN", -0.9544997f0, "0:4", "6:11 6:11 4:11 9:11"), Match(145267, 137193, "JPN", "KOR", 0.9544997f0, "4:0", "11:8 11:7 11:5 11:2"), Match(203067, 135023, "CHN", "ROU", 0.6826895f0, "4:2", "5:11 18:20 11:9 11:8 11:9 11:7"), Match(137237, 132824, "CHN", "TPE", 0.6826895f0, "4:2", "8:11 5:11 13:11 11:3 11:4 11:2"), Match(134011, 137465, "ROU", "CHN", 0.6826895f0, "4:2", "11:9 13:11 10:12 11:6 8:11 11:9"), Match(135049, 135493, "CHN", "GER", 0.8663856f0, "4:1", "11:6 11:8 5:11 16:14 13:11"), Match(137630, 203029, "POR", "CHN", -0.9544997f0, "0:4", "5:11 9:11 9:11 9:11"), Match(204289, 143604, "CHN", "JPN", -0.3829249f0, "3:4", "7:11 9:11 11:9 6:11 11:8 11:5 6:11")  …  Match(200255, 146262, "SLO", "AUT"

In [1]:
function test()
    file_path = "matches/matches_4.json"
    
    new_match_count = 0
    open(file_path, "r") do file
        matches_data = JSON.parse(file)
    
                
        # ITTF格式是数组的数组
        for match_list in matches_data
            for raw_match in match_list
                event_id = raw_match["vw_matches___tournament_id_raw"]
                
                match = convert_ittf_match(raw_match)
                
                # 只添加有效的单打比赛
                if !isnothing(match)
                    println(match)
                    # push!(events_dict[event_id].match, match)
                    new_match_count += 1
                end
            end
        end
    end

    new_match_count
end

test (generic function with 1 method)

In [6]:
test()

Match(137237, 203067, "CHN", "CHN", 0.8663856f0, "4:1", "6:11 11:6 11:9 11:2 11:3")
Match(135049, 134011, "CHN", "ROU", 0.9544997f0, "4:0", "11:3 13:11 11:5 11:5")
Match(143604, 203029, "JPN", "CHN", -0.9544997f0, "0:4", "6:11 6:11 4:11 9:11")
Match(145267, 137193, "JPN", "KOR", 0.9544997f0, "4:0", "11:8 11:7 11:5 11:2")
Match(203067, 135023, "CHN", "ROU", 0.6826895f0, "4:2", "5:11 18:20 11:9 11:8 11:9 11:7")
Match(137237, 132824, "CHN", "TPE", 0.6826895f0, "4:2", "8:11 5:11 13:11 11:3 11:4 11:2")
Match(134011, 137465, "ROU", "CHN", 0.6826895f0, "4:2", "11:9 13:11 10:12 11:6 8:11 11:9")
Match(135049, 135493, "CHN", "GER", 0.8663856f0, "4:1", "11:6 11:8 5:11 16:14 13:11")
Match(137630, 203029, "POR", "CHN", -0.9544997f0, "0:4", "5:11 9:11 9:11 9:11")
Match(204289, 143604, "CHN", "JPN", -0.3829249f0, "3:4", "7:11 9:11 11:9 6:11 11:8 11:5 6:11")
Match(145035, 137193, "TPE", "KOR", -0.6826895f0, "2:4", "11:8 7:11 7:11 11:9 9:11 7:11")
Match(145267, 202970, "JPN", "CHN", 0.3829249f0, "4:3",

86

In [9]:
players[135937]

Player(135937, "TAKAMORI Mao", "W", Dict("JPN" => Date("2025-04-13")), 2008, "Right-Hand", "Attack", "ShakeHand")

In [13]:
for m in events[Date("2023-12-03")][1].match
    if m.player_a_id == 135937
        println(m)
    end
end

Match(135937, 203618, "JPN", "CHN", -0.8663856f0, "1:4", "11:0 7:11 5:11 4:11 7:11")
Match(135937, 200903, "JPN", "SLO", 0.9544997f0, "4:0", "11:2 11:7 11:1 11:1")
Match(135937, 203618, "JPN", "CHN", 0.80606914f0, "3:0", "11:5 11:7 11:7")
Match(135937, 202970, "JPN", "CHN", 0.33499447f0, "3:2", "9:11 7:11 12:10 11:9 11:8")
Match(135937, 145035, "JPN", "TPE", 0.80606914f0, "3:0", "11:2 11:9 11:3")
